# Structured Outputs with llm_generate using the vLLM Provider

Dont forget to follow installation and setup instructions in the readme.

WARNING: vLLM has experimental support for macOS with Apple Silicon. For now, users must build from source to natively run on macOS.

Colab or Intel/AMD Recommended for CPU. 


#### Authenticate with HuggingFace for Access to Gemma-3 Series models

In [ ]:
!hf auth login

In [ ]:
import daft
from daft import col
from daft.functions import llm_generate, format
from vllm.sampling_params import GuidedDecodingParams

MAX_TOKENS = 100
TEMPERATURE = 0.0

## Guided Choice

In [12]:
df_choice = daft.from_pydict({
    "text": [
        "I'm not a fan of slow data pipelines",
        "Daft Dataframes are wicked fast!",
    ]
})

df_choice = df_choice.with_column("sentiment", llm_generate(
    "Classify this sentiment: " + df_choice["text"],
    #model="google/gemma-3-270m",
    provider="vllm",
    #guided_decoding=GuidedDecodingParams(choice=["Positive", "Negative"]),
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
)).collect()
df_choice.show()

                                                          d





















Error when running pipeline node UDF _vLLMGenerator


RuntimeError: Actor Pool UDF unexpectedly failed with traceback:
Traceback (most recent call last):

  File "/Users/everett-founder/git/ugh/daft-structured-outputs/.venv/lib/python3.12/site-packages/daft/execution/udf_worker.py", line 48, in udf_event_loop
    initialized_projection = ExpressionsProjection([e._initialize_udfs() for e in uninitialized_projection])
                                                    ^^^^^^^^^^^^^^^^^^^^

  File "/Users/everett-founder/git/ugh/daft-structured-outputs/.venv/lib/python3.12/site-packages/daft/expressions/expressions.py", line 1952, in _initialize_udfs
    return Expression._from_pyexpr(initialize_udfs(self._expr))
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/everett-founder/git/ugh/daft-structured-outputs/.venv/lib/python3.12/site-packages/daft/udf/legacy.py", line 41, in initialize
    return self.inner(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/everett-founder/git/ugh/daft-stru

## Guided Regex

In [ ]:
df_regex = daft.from_pydict({
    "name": [
        "John Doe",
        "Jane Smith",
        "Alice Johnson",
        "Bob Brown",
        "Charlie Davis",
    ],
    "company": [
        "Acme Inc.",
        "Globex Corp.",
        "Initech",
        "Soylent Corp.",
        "Umbrella Corp.",
    ]
}) 

df_regex = df_regex.with_column("email", llm_generate(
    format("""
    Generate an email address for {} at {}
    End in .com and new line.
    Example result:
    alan.turing@enigma.com
    """, col["name"], col["company"]),
    model="google/gemma-3-270m",
    provider="vllm",
    guided_decoding=GuidedDecodingParams(regex=r"\w+@\w+\.com\n"),
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
))
df_regex.show()


## Guided Decoding by Pydantic JSON Schema

In [ ]:
from pydantic import BaseModel
from enum import Enum

class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"


class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType


df_pydantic = daft.from_pydict({
    "decade": [
        "80's",
        "90's",
        "2000's",
    ]
})

df_pydantic = df_pydantic.with_column("car_description_json", llm_generate(
    format("Generate a car description for the {} decade", col["decade"]),
    model="google/gemma-3-270m",
    provider="vllm",
    guided_decoding=GuidedDecodingParams(json=CarDescription.model_json_schema()),
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
))

df_pydantic = df_pydantic.with_column("car_description_validated", 
    df_pydantic["car_description_json"].apply(
        lambda x: CarDescription.model_validate_json(x),
        return_dtype= daft.DataType.python()
    )
)
df_pydantic.show()
        

## Guided Decoding by Grammar

In [ ]:
simplified_sql_grammar = """
root ::= select_statement
select_statement ::= "SELECT " column " from " table " where " condition
column ::= "col_1 " | "col_2 "
table ::= "table_1 " | "table_2 "
condition ::= column "= " number
number ::= "1 " | "2 "
limit ::= "LIMIT " number
"""

df_grammar = daft.from_pydict({
    "prompt": [
        "Generate an SQL query to show the 'username' and 'email' from the 'users' table.",
        "Generate an SQL query to show the 'name' and 'age' from the 'users' table where the age is greater than 30.",
        "Generate an SQL query to show the 'name' and 'age' from the 'users' table where the age is greater than 30 and the name is 'John Doe'.",
        "Show me how to see the first 10 rows of the 'users' table.",
    ]
})

df_grammar = df_grammar.with_column("sql_query", llm_generate(
    df_grammar["prompt"],
    model="google/gemma-3-270m",
    provider="vllm",
    guided_decoding=GuidedDecodingParams(grammar=simplified_sql_grammar),
))
df_grammar.show()